In [141]:
import requests
import pandas as pd
from google.cloud import bigquery
from datetime import datetime
from google.oauth2 import service_account

TESTES PARA EXTRAÇÃO DE DADOS, TESTE DE ACESSO DAS CONTAS NO GCP E TESTE DE INSERÇÃO DE DADOS NO BIGQUERY.

ACESSA A TABELA DESEJADA. SÓ É NECESSÁRIO ENTRAR NA TABELA PELO SITE DA ANEEL, ENCONTRAR A TABELA DESEJADA E, ATRAVÉS DA EXPORTAÇÃO VIA API, ENCONTRAR O  "RESOURCE_ID" E SUBSTITUIR.

ALÉM DISSO, NESTA ETAPA É FEITA A PAGINAÇÃO DA REQUISIÇÃO, TENDO EM VISTA QUE A REQUISIÇÃO SE LIMITA A 32000 LINHAS POR PÁGINA, ENTÃO É NECESSÁRIO QUE SEJA FEITO UM PROCESSO DE PAGINAÇÃO PARA EFETUAR MÚLTIPLAS REQUISIÇÕES E ARMAZENÁ-LAS NA LISTA. ESTA SENDO EXIBIDA POSTERIORMENTE NO DATAFRAME.

In [142]:
import requests
import pandas as pd

# URL base da API
base_url = "https://dadosabertos.aneel.gov.br/api/3/action/datastore_search"

# ID do recurso (substitua pelo seu ID de recurso)
resource_id = "cce72c2b-a4cb-4b98-818f-3ac56bbf2a78"

# Número máximo de resultados por página
limit_per_page = 32000  # Defina o limite máximo

# Inicializa uma lista para armazenar todos os dados
all_data = []

# Inicializa o parâmetro de offset para a primeira página
offset = 0

while True:
    # Parâmetros da requisição
    params = {
        "resource_id": resource_id,
        "limit": limit_per_page,
        "offset": offset,
        "q": "EQUATORIAL ALAGOAS DISTRIBUIDORA DE ENERGIA S.A."
    }

    # Faz a requisição à API
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        records = data['result']['records']
        
        # Adiciona os registros da página à lista de todos os dados
        all_data.extend(records)
        
        # Verifica se há mais páginas
        if len(records) < limit_per_page:
            break  # Sai do loop se não houver mais páginas
        else:
            offset += limit_per_page  # Avança para a próxima página
    else:
        print("Falha na requisição:", response.status_code)
        break

# Cria um DataFrame com todos os dados
df = pd.DataFrame(all_data)

# Agora você tem todos os dados em um DataFrame
df

,_id,DatGeracaoConjuntoDados,DscConjuntoUnidadeConsumidora,DscAlimentadorSubestacao,DscSubestacaoDistribuicao,NumOrdemInterrupcao,DscTipoInterrupcao,IdeMotivoInterrupcao,DatInicioInterrupcao,DatFimInterrupcao,DscFatoGeradorInterrupcao,NumNivelTensao,NumUnidadeConsumidora,NumConsumidorConjunto,NumAno,NomAgenteRegulado,SigAgente,NumCPFCNPJ,rank
0,961748,2023-09-10,MATRIZ DE CAMARAGIBE,-,-,2023-4-28339-12637336,Não Programada,0,2023-04-27 14:55:42,2023-04-27 18:46:00,INTERNA;NAO PROGRAMADA;TERCEIROS;INTERFERENCIA...,380,1,18492,2023,EQUATORIAL ALAGOAS DISTRIBUIDORA DE ENERGIA S.A.,Equatorial AL,12272084000100,0.475594
1,972919,2023-09-10,PÃO DE AÇÚCAR,-,-,2023-5-28263-12669780,Não Programada,0,2023-05-26 13:03:19,2023-05-26 16:27:00,INTERNA;NAO PROGRAMADA;TERCEIROS;INTERFERENCIA...,380,1,10767,2023,EQUATORIAL ALAGOAS DISTRIBUIDORA DE ENERGIA S.A.,Equatorial AL,12272084000100,0.475594
2,974825,2023-09-10,MATRIZ DE CAMARAGIBE,Y1,MTC,2023-5-34307-12675784,Não Programada,0,2023-05-31 16:49:39,2023-05-31 17:35:00,INTERNA;NAO PROGRAMADA;TERCEIROS;INTERFERENCIA...,380,1,18633,2023,EQUATORIAL ALAGOAS DISTRIBUIDORA DE ENERGIA S.A.,Equatorial AL,12272084000100,0.475593
3,975156,2023-09-10,PÃO DE AÇÚCAR,Y3,PDA,2023-6-504-12676602,Não Programada,0,2023-06-01 11:03:43,2023-06-02 10:03:00,INTERNA;NAO PROGRAMADA;TERCEIROS;INTERFERENCIA...,380,1,10715,2023,EQUATORIAL ALAGOAS DISTRIBUIDORA DE ENERGIA S.A.,Equatorial AL,12272084000100,0.475593
4,961559,2023-09-10,MATRIZ DE CAMARAGIBE,Y1,MTC,2023-4-27789-12636786,Não Programada,0,2023-04-27 08:35:15,2023-04-27 09:56:00,INTERNA;NAO PROGRAMADA;TERCEIROS;INTERFERENCIA...,380,1,18492,2023,EQUATORIAL ALAGOAS DISTRIBUIDORA DE ENERGIA S.A.,Equatorial AL,12272084000100,0.475593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84149,942019,2023-09-10,MARIBONDO,Y3,MBD,2023-3-6312-12580190,Não Programada,0,2023-03-07 18:20:25,2023-03-07 19:18:00,INTERNA;NAO PROGRAMADA;MEIO AMBIENTE;ANIMAIS,13800,13,19836,2023,EQUATORIAL ALAGOAS DISTRIBUIDORA DE ENERGIA S.A.,Equatorial AL,12272084000100,0.465475
84150,939831,2023-09-10,MARITUBA,Y2,MTB,2023-3-23986-12569693,Programada,0,2023-03-01 10:05:00,2023-03-01 12:30:00,INTERNA;PROGRAMADA;MANUTENCAO;PREVENTIVA,13800,894,42044,2023,EQUATORIAL ALAGOAS DISTRIBUIDORA DE ENERGIA S.A.,Equatorial AL,12272084000100,0.465475
84151,943480,2023-09-10,BENEDITO BENTES,Y6,BBE,2023-3-10741-12585427,Não Programada,0,2023-03-11 18:03:35,2023-03-11 19:20:00,INTERNA;NAO PROGRAMADA;MEIO AMBIENTE;QUEIMA OU...,380,1,87183,2023,EQUATORIAL ALAGOAS DISTRIBUIDORA DE ENERGIA S.A.,Equatorial AL,12272084000100,0.465475
84152,939833,2023-09-10,MARITUBA,Y3,MTB,2023-3-326-12574265,Não Programada,0,2023-03-01 10:10:31,2023-03-01 11:20:00,INTERNA;NAO PROGRAMADA;PROPRIAS DO SISTEMA;NAO...,380,9,42044,2023,EQUATORIAL ALAGOAS DISTRIBUIDORA DE ENERGIA S.A.,Equatorial AL,12272084000100,0.465475


NESTA ETAPA SÃO VISUALIZADOS OS DATASETS EM QUE A SERVICE ACCOUNT TEM ACESSO

In [143]:


# Substitua pelo caminho para o seu arquivo de credenciais JSON
credentials_path = "gcp-credentials.json"

# Cria uma instância do cliente BigQuery
client = bigquery.Client.from_service_account_json(credentials_path)

# Lista os datasets no projeto
datasets = list(client.list_datasets())
if datasets:
    print("Datasets disponíveis:")
    for dataset in datasets:
        print(f"- {dataset.dataset_id}")
else:
    print("Nenhum dataset encontrado.")

# Realize outras operações de teste aqui, se desejar

Datasets disponíveis:
- raw
- raw2


AGORA, AO PASSAR AS INFORMAÇÕES DE PROJECT_ID, DATASET_ID E TABLE_ID, É POSSÍVEL VISUALIZAR SE SUA SERVICE ACCOUNT TEM ACESSO À TABELA EM QUESTÃO

In [144]:

# Substitua pelo caminho para o seu arquivo de credenciais JSON
credentials_path = "gcp-credentials.json"

# Configura as credenciais
client = bigquery.Client.from_service_account_json(credentials_path)

# Substitua pelos detalhes da sua tabela e projeto
project_id = 'novo-projeto-400016'
dataset_id = 'raw'
table_id = 'tab_teste'

# Verifica se a tabela existe e se você tem permissão para acessá-la
try:
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    table = client.get_table(table_ref)
    
    # Se você chegou aqui, a tabela existe e você tem permissão para acessá-la
    print(f'Você tem acesso à tabela {table_id}.')
except Exception as e:
    # Se ocorrer um erro, você não tem acesso à tabela
    print(f'Você não tem acesso à tabela {table_id}. Erro: {str(e)}')

Você tem acesso à tabela tab_teste.


POR FIM, É FEITO O TESTE DE INSERÇÃO NA TABELA DESEJADA.

In [146]:
# Substitua pelo caminho para o seu arquivo de credenciais JSON
credentials_path = "gcp-credentials.json"

# Configura as credenciais
client = bigquery.Client.from_service_account_json(credentials_path)

# Substitua pelos detalhes da sua tabela e projeto
project_id = 'novo-projeto-400016'
dataset_id = 'raw'
table_id = 'tab_interrupcoes'

# Insere os dados no BigQuery
table_ref = client.dataset(dataset_id).table(table_id)
job_config = bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE', autodetect=True)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  # Aguarda a conclusão do job

print(f'Dados inseridos na tabela {table_id}')

Dados inseridos na tabela tab_interrupcoes


Exemplo de requisição JSON para a função:

 {
     "base_url": ""
     
     "resource_id": "",

     "limit_per_page": 0,

     "project_id": "",

     "dataset_id": "",
     
     "table_id": ""
 }